In [66]:
from offres_emploi import Api
from offres_emploi.utils import dt_to_str_iso
import datetime
import pandas as pd

pd.set_option("display.max_columns", None)

In [69]:
# Fonction pour effectuer une recherche avec gestion d'erreurs
def clean_dict_columns(df: pd.DataFrame):
    df['dateActualisation'] = pd.to_datetime(df['dateActualisation'])
    df['dateCreation'] = pd.to_datetime(df['dateCreation'])
    df_lieu_travail = pd.json_normalize(df["lieuTravail"])
    df_lieu_travail.drop(["latitude", "longitude"], axis=1, inplace=True)
    df_entreprise = pd.json_normalize(df["entreprise"])
    df_entreprise.drop(["entrepriseAdaptee", 'url', 'logo'], axis=1, inplace=True)
    df_salaire = pd.json_normalize(df["salaire"])
    df_salaire.drop(["complement1", "complement2", "commentaire"], axis=1, inplace=True)
    df_formations = df["formations"].explode()
    df_formations = pd.json_normalize(df_formations)
    df_formations.drop(["codeFormation", "domaineLibelle", "exigence", "commentaire"], axis=1, inplace=True)
    df_final = pd.concat([df, df_formations, df_salaire, df_entreprise, df_lieu_travail], axis=1)
    return df_final

def job_offers_pole_emploi(api_client, params, cols_to_drop):
    # Initialisation des variables locales
    results = []
    start_range = 0
    max_results = float('inf')

    try:
        # Pagination pour récupérer tous les résultats
        while start_range < max_results:
            params['range'] = f"{start_range}-{start_range + 149}"  # Utilisation de la pagination

            # Effectuer la recherche
            search_result = api_client.search(params=params)
            content_range = search_result.get('Content-Range', {})
            max_results_str = content_range.get('max_results', '0')
            max_results = int(max_results_str)

            # Ajouter les résultats actuels à la liste globale
            results.extend(search_result.get('resultats', []))

            # Mettre à jour la plage de départ pour la prochaine itération
            start_range += 150

        if max_results > 0:
            df_emploi = pd.DataFrame(results)
            df_final = clean_dict_columns(df_emploi)
            df_final.drop(cols_to_drop, axis=1, inplace=True)
            return df_final

        else:
            print("Aucune offre d'emploi trouvée.")
            return pd.DataFrame()

    except Exception as e:
        print(f"Une erreur s'est produite lors de la recherche : {e}")


In [70]:
# Configuration du client API
client = Api(client_id="PAR_datajobs_addbc0bc41d7d51f05b218a78c5a95e14be4d73d536fde31c5962de09420f7ba",
             client_secret="ec561083589b4912fb4feebf33ef1078098c0b8b3f8ff04a665d3e46173f11e1")

# Exemple de recherche plus complexe
start_dt = datetime.datetime(2023, 9, 1, 12, 30)
end_dt = datetime.datetime.today()
params = {
    "motsCles": "data analyst",
    'minCreationDate': dt_to_str_iso(start_dt),
    'maxCreationDate': dt_to_str_iso(end_dt)
}

# Appel de la fonction de recherche avec gestion des erreurs
df_emploi_result = perform_search(client, params)

df_emploi_result.to_csv("df_emploi_result.csv", index=False)

In [71]:
df_emploi_result

,intitule,description,dateCreation,dateActualisation,typeContrat,natureContrat,competences,dureeTravailLibelle,dureeTravailLibelleConverti,secteurActiviteLibelle,niveauLibelle,libelle,nom,description,libelle,codePostal,commune
0,Data analyst,"Vous aussi, vous pensez que tout est mesurable...",2023-12-18 14:45:32+00:00,2023-12-19 10:59:04+00:00,CDI,Contrat travail,"[{'code': '109527', 'libelle': 'Adapter les ou...",35H Travail en journée,Temps plein,Conseil en systèmes et logiciels informatiques,Bac+2 ou équivalents,"Annuel de 35000,00 Euros sur 12 mois",OTCHEE,NaN,92 - COURBEVOIE,92400,92026
1,Data Analyst (H/F),Nous recherchons pour notre client spécialisé ...,2023-12-18 11:36:50+00:00,2023-12-19 14:51:10+00:00,CDI,Contrat travail,"[{'code': '109527', 'libelle': 'Adapter les ou...",35H Travail en journée,Temps plein,Activités des agences de placement de main-d'œ...,Bac+2 ou équivalents,"Mensuel de 2500,00 Euros sur 12 mois",NaN,NaN,92 - BAGNEUX,92220,92007
2,Data Analyst expérimenté (H/F),"Chez K-LAGAN, nous parions sur vous !\n\nVous ...",2023-12-18 09:54:33+00:00,2023-12-19 14:52:22+00:00,CDI,Contrat travail,"[{'code': '100113', 'libelle': 'Logiciels de g...",38H30 Travail en journée,Temps plein,"Ingénierie, études techniques",NaN,"Annuel de 42000,00 Euros à 52000,00 Euros sur ...",SAS K-LAGAN FRANCE,NaN,Ile-de-France,NaN,NaN
3,DATA BUSINESS ANALYST (H/F),Vous êtes un(e) Data Business Analyst confirmé...,2023-12-15 15:42:50+00:00,2023-12-19 15:20:17+00:00,CDI,Contrat travail,"[{'code': '109527', 'libelle': 'Adapter les ou...",37H30 Travail en journée,Temps plein,Programmation informatique,Bac+5 et plus ou équivalents,NaN,HARDIS GROUPE,"""Hardis Group est une ESN, une entreprise de c...",38 - GRENOBLE,38000,38185
4,Data Analyst (H/F),Nous recherchons pour notre client spécialisé ...,2023-12-15 11:34:43+00:00,2023-12-19 14:56:58+00:00,CDI,Contrat travail,"[{'code': '109527', 'libelle': 'Adapter les ou...",35H Travail en journée,Temps plein,Activités des agences de placement de main-d'œ...,Bac+2 ou équivalents,"Mensuel de 2500,00 Euros sur 12 mois",NaN,NaN,92 - NANTERRE,92000,92050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Responsable équipe Risk Analytics (F/H) (H/F),Nous recherchons un(e) :\n\n \n\nData Analyste...,2023-12-13 18:24:03+00:00,2023-12-13 18:24:03+00:00,CDI,Contrat travail,NaN,NaN,NaN,Autres assurances,NaN,NaN,NaN,Carrefour Assurance a pour vocation depuis plu...,91 - MASSY,91300,91377
274,Analyste Qualité des Données (F/H) (H/F),Poste et missions\n\nVous serez rattaché.e à l...,2023-12-06 18:55:20+00:00,2023-12-06 18:55:20+00:00,CDI,Contrat travail,NaN,NaN,NaN,NaN,NaN,NaN,BPCE SEF,"BPCE Financement, c'est rejoindre le Groupe BP...",75 - PARIS 13,75013,75113
275,TECH LEAD (H/F),"Dans ce contexte, nous vous proposons l'opport...",2023-11-27 15:33:18+00:00,2023-11-27 15:33:18+00:00,CDI,Contrat travail,NaN,NaN,NaN,Autres activités informatiques,NaN,NaN,NaN,"Nous rejoindre, c'est rejoindre l'un des leade...",91 - MASSY,91300,91377
276,Analyste BI (H/F),RESPONSABILITÉS : \n\nVos missions ?\nIntégré ...,2023-11-26 02:43:00+00:00,2023-11-26 02:43:00+00:00,CDI,Contrat travail,NaN,35 H Travail en journée,Temps plein,NaN,NaN,"Annuel de 30000,00 Euros à 45000,00 Euros",Amiltone,Qui sommes-nous ?\nNous sommes passionnés par ...,42 - ST ETIENNE,42000,42218


# Tests

In [57]:
df_lieu_travail = pd.json_normalize(df_emploi_result["lieuTravail"])
df_lieu_travail.drop(["latitude", "longitude", "commune"], axis=1, inplace=True)
df_lieu_travail

,libelle,codePostal,commune
0,92 - COURBEVOIE,92400,92026
1,92 - BAGNEUX,92220,92007
2,Ile-de-France,NaN,NaN
3,38 - GRENOBLE,38000,38185
4,92 - NANTERRE,92000,92050
...,...,...,...
273,91 - MASSY,91300,91377
274,75 - PARIS 13,75013,75113
275,91 - MASSY,91300,91377
276,42 - ST ETIENNE,42000,42218


In [58]:
df_entreprise = pd.json_normalize(df_emploi_result["entreprise"])
df_entreprise.drop(["entrepriseAdaptee", 'url', 'logo'], axis=1, inplace=True)
df_entreprise

,nom,description
0,OTCHEE,NaN
1,NaN,NaN
2,SAS K-LAGAN FRANCE,NaN
3,HARDIS GROUPE,"""Hardis Group est une ESN, une entreprise de c..."
4,NaN,NaN
...,...,...
273,NaN,Carrefour Assurance a pour vocation depuis plu...
274,BPCE SEF,"BPCE Financement, c'est rejoindre le Groupe BP..."
275,NaN,"Nous rejoindre, c'est rejoindre l'un des leade..."
276,Amiltone,Qui sommes-nous ?\nNous sommes passionnés par ...


In [59]:
df_salaire = pd.json_normalize(df_emploi_result["salaire"])
df_salaire.drop(["complement1", "complement2", "commentaire"], axis=1, inplace=True)
df_salaire

,libelle
0,"Annuel de 35000,00 Euros sur 12 mois"
1,"Mensuel de 2500,00 Euros sur 12 mois"
2,"Annuel de 42000,00 Euros à 52000,00 Euros sur ..."
3,NaN
4,"Mensuel de 2500,00 Euros sur 12 mois"
...,...
273,NaN
274,NaN
275,NaN
276,"Annuel de 30000,00 Euros à 45000,00 Euros"


In [60]:
df_formations = df_emploi_result["formations"].explode()
df_formations = pd.json_normalize(df_formations)
df_formations.drop(["codeFormation", "domaineLibelle", "exigence", "commentaire"], axis=1, inplace=True)
df_formations

,niveauLibelle
0,Bac+2 ou équivalents
1,Bac+2 ou équivalents
2,NaN
3,Bac+5 et plus ou équivalents
4,Bac+2 ou équivalents
...,...
273,NaN
274,NaN
275,NaN
276,NaN


In [61]:
df_emploi_result.drop(["id", "lieuTravail", "romeCode", "romeLibelle", "origineOffre", "appellationlibelle", "entreprise", "typeContratLibelle", "experienceExige", "experienceLibelle", "formations", "langues", "salaire", "alternance", "contact", "nombrePostes", "accessibleTH", "deplacementCode", "deplacementLibelle", "qualificationCode", "qualificationLibelle", "codeNAF", "secteurActivite", "qualitesProfessionnelles", "offresManqueCandidats", "experienceCommentaire", "permis", "complementExercice"], axis=1, inplace=True)

In [62]:
final = pd.concat([df_emploi_result, df_formations, df_salaire, df_entreprise, df_lieu_travail], axis=1)
final

,intitule,description,dateCreation,dateActualisation,typeContrat,natureContrat,competences,dureeTravailLibelle,dureeTravailLibelleConverti,secteurActiviteLibelle,niveauLibelle,libelle,nom,description,libelle,codePostal,commune
0,Data analyst,"Vous aussi, vous pensez que tout est mesurable...",2023-12-18 14:45:32+00:00,2023-12-19 10:59:04+00:00,CDI,Contrat travail,"[{'code': '109527', 'libelle': 'Adapter les ou...",35H Travail en journée,Temps plein,Conseil en systèmes et logiciels informatiques,Bac+2 ou équivalents,"Annuel de 35000,00 Euros sur 12 mois",OTCHEE,NaN,92 - COURBEVOIE,92400,92026
1,Data Analyst (H/F),Nous recherchons pour notre client spécialisé ...,2023-12-18 11:36:50+00:00,2023-12-19 14:51:10+00:00,CDI,Contrat travail,"[{'code': '109527', 'libelle': 'Adapter les ou...",35H Travail en journée,Temps plein,Activités des agences de placement de main-d'œ...,Bac+2 ou équivalents,"Mensuel de 2500,00 Euros sur 12 mois",NaN,NaN,92 - BAGNEUX,92220,92007
2,Data Analyst expérimenté (H/F),"Chez K-LAGAN, nous parions sur vous !\n\nVous ...",2023-12-18 09:54:33+00:00,2023-12-19 14:52:22+00:00,CDI,Contrat travail,"[{'code': '100113', 'libelle': 'Logiciels de g...",38H30 Travail en journée,Temps plein,"Ingénierie, études techniques",NaN,"Annuel de 42000,00 Euros à 52000,00 Euros sur ...",SAS K-LAGAN FRANCE,NaN,Ile-de-France,NaN,NaN
3,DATA BUSINESS ANALYST (H/F),Vous êtes un(e) Data Business Analyst confirmé...,2023-12-15 15:42:50+00:00,2023-12-19 15:20:17+00:00,CDI,Contrat travail,"[{'code': '109527', 'libelle': 'Adapter les ou...",37H30 Travail en journée,Temps plein,Programmation informatique,Bac+5 et plus ou équivalents,NaN,HARDIS GROUPE,"""Hardis Group est une ESN, une entreprise de c...",38 - GRENOBLE,38000,38185
4,Data Analyst (H/F),Nous recherchons pour notre client spécialisé ...,2023-12-15 11:34:43+00:00,2023-12-19 14:56:58+00:00,CDI,Contrat travail,"[{'code': '109527', 'libelle': 'Adapter les ou...",35H Travail en journée,Temps plein,Activités des agences de placement de main-d'œ...,Bac+2 ou équivalents,"Mensuel de 2500,00 Euros sur 12 mois",NaN,NaN,92 - NANTERRE,92000,92050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Responsable équipe Risk Analytics (F/H) (H/F),Nous recherchons un(e) :\n\n \n\nData Analyste...,2023-12-13 18:24:03+00:00,2023-12-13 18:24:03+00:00,CDI,Contrat travail,NaN,NaN,NaN,Autres assurances,NaN,NaN,NaN,Carrefour Assurance a pour vocation depuis plu...,91 - MASSY,91300,91377
274,Analyste Qualité des Données (F/H) (H/F),Poste et missions\n\nVous serez rattaché.e à l...,2023-12-06 18:55:20+00:00,2023-12-06 18:55:20+00:00,CDI,Contrat travail,NaN,NaN,NaN,NaN,NaN,NaN,BPCE SEF,"BPCE Financement, c'est rejoindre le Groupe BP...",75 - PARIS 13,75013,75113
275,TECH LEAD (H/F),"Dans ce contexte, nous vous proposons l'opport...",2023-11-27 15:33:18+00:00,2023-11-27 15:33:18+00:00,CDI,Contrat travail,NaN,NaN,NaN,Autres activités informatiques,NaN,NaN,NaN,"Nous rejoindre, c'est rejoindre l'un des leade...",91 - MASSY,91300,91377
276,Analyste BI (H/F),RESPONSABILITÉS : \n\nVos missions ?\nIntégré ...,2023-11-26 02:43:00+00:00,2023-11-26 02:43:00+00:00,CDI,Contrat travail,NaN,35 H Travail en journée,Temps plein,NaN,NaN,"Annuel de 30000,00 Euros à 45000,00 Euros",Amiltone,Qui sommes-nous ?\nNous sommes passionnés par ...,42 - ST ETIENNE,42000,42218


In [12]:
df_emploi_result

,id,intitule,description,dateCreation,dateActualisation,lieuTravail,romeCode,romeLibelle,appellationlibelle,entreprise,typeContrat,typeContratLibelle,natureContrat,experienceExige,experienceLibelle,formations,langues,competences,salaire,dureeTravailLibelle,dureeTravailLibelleConverti,alternance,contact,nombrePostes,accessibleTH,deplacementCode,deplacementLibelle,qualificationCode,qualificationLibelle,codeNAF,secteurActivite,secteurActiviteLibelle,qualitesProfessionnelles,origineOffre,offresManqueCandidats,experienceCommentaire,permis,complementExercice
0,166MBJK,Data analyst,"Vous aussi, vous pensez que tout est mesurable...",2023-12-18 14:45:32+00:00,2023-12-19 10:59:04+00:00,"{'libelle': '92 - COURBEVOIE', 'latitude': 48....",M1403,Études et prospectives socio-économiques,Data analyst,"{'nom': 'OTCHEE', 'entrepriseAdaptee': False}",CDI,Contrat à durée indéterminée,Contrat travail,E,2 ans,"[{'codeFormation': '31054', 'domaineLibelle': ...","[{'libelle': 'Anglais', 'exigence': 'S'}]","[{'code': '109527', 'libelle': 'Adapter les ou...","{'libelle': 'Annuel de 35000,00 Euros sur 12 m...",35H Travail en journée,Temps plein,False,"{'nom': 'OTCHEE - Mme Alexandra DELEDICQ', 'co...",1,False,3,Fréquents Zone régionale,9,Cadre,62.02A,62,Conseil en systèmes et logiciels informatiques,"[{'libelle': 'Travailler en équipe', 'descript...","{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN
1,166LRMP,Data Analyst (H/F),Nous recherchons pour notre client spécialisé ...,2023-12-18 11:36:50+00:00,2023-12-19 14:15:00+00:00,"{'libelle': '92 - BAGNEUX', 'latitude': 48.797...",M1403,Études et prospectives socio-économiques,Data analyst,{'entrepriseAdaptee': False},CDI,Contrat à durée indéterminée,Contrat travail,E,2 ans,"[{'niveauLibelle': 'Bac+2 ou équivalents', 'ex...",NaN,"[{'code': '109527', 'libelle': 'Adapter les ou...","{'libelle': 'Mensuel de 2500,00 Euros sur 12 m...",35H Travail en journée,Temps plein,False,{},1,False,1,Jamais,8,Agent de maîtrise,78.10Z,78,Activités des agences de placement de main-d'œ...,[{'libelle': 'Prendre des initiatives et être ...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN
2,166LJPL,Data Analyst expérimenté (H/F),"Chez K-LAGAN, nous parions sur vous !\n\nVous ...",2023-12-18 09:54:33+00:00,2023-12-19 11:17:58+00:00,{'libelle': 'Ile-de-France'},M1403,Études et prospectives socio-économiques,Data analyst,"{'nom': 'SAS K-LAGAN FRANCE', 'entrepriseAdapt...",CDI,Contrat à durée indéterminée,Contrat travail,E,5 ans,NaN,"[{'libelle': 'Espagnol', 'exigence': 'S'}]","[{'code': '100113', 'libelle': 'Logiciels de g...","{'libelle': 'Annuel de 42000,00 Euros à 52000,...",38H30 Travail en journée,Temps plein,False,{},1,False,4,Quotidiens Zone régionale,9,Cadre,71.12B,71,"Ingénierie, études techniques",[{'libelle': 'Prendre des initiatives et être ...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN
3,166KMJK,DATA BUSINESS ANALYST (H/F),Vous êtes un(e) Data Business Analyst confirmé...,2023-12-15 15:42:50+00:00,2023-12-18 09:15:02+00:00,"{'libelle': '38 - GRENOBLE', 'latitude': 45.18...",M1403,Études et prospectives socio-économiques,Data analyst,"{'nom': 'HARDIS GROUPE', 'description': '""Hard...",CDI,Contrat à durée indéterminée,Contrat travail,E,5 ans,[{'niveauLibelle': 'Bac+5 et plus ou équivalen...,"[{'libelle': 'Anglais', 'exigence': 'S'}]","[{'code': '109527', 'libelle': 'Adapter les ou...","{'commentaire': 'à négocier', 'complement1': '...",37H30 Travail en journée,Temps plein,False,{'nom': 'HARDIS GROUPE - Mme Delphine CORREARD...,1,False,1,Jamais,9,Cadre,62.01Z,62,Programmation informatique,[{'libelle': 'Prendre des initiatives et être ...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN
4,166KBFG,Data Analyst (H/F),Nous recherchons pour notre client spécialisé ...,2023-12-15 11:34:43+00:00,2023-12-19 13:34:30+00:00,"{'libelle': '92 - NANTERRE', 'latitude': 48.89...",M1403,Études et prospectives socio-économiques,Data analyst,{'entr

In [5]:
# Enregistrez le DataFrame dans un fichier CSV
df_emploi_result.to_csv("df_emploi_result.csv", index=False, encoding='utf-8', sep=';')